## In this assignment, we will get acquainted with essential matrix and fundamental matrix. And recover the essential matrix from matched points of two scene with OpenCV. Furthermore, we will try to localize these matched points in the 3D space


In [1]:
# Read matched points from two scenes

import pickle
with open('data_points.pickle','rb') as handle:
    data=pickle.load(handle)

In [2]:
# Find fundamental matrix from the match points using OpenCV
import cv2

pts1 = data['x1']
pts2 = data['x2']

F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_8POINT)
pts1 = pts1[mask.ravel()==1] # remove outliers by only keeping inliers
pts2 = pts2[mask.ravel()==1] # 

In [3]:
# "Find" essential matrix
import numpy as np

K = np.eye(3) # Take the camera intrisic matrix matrix K to be identity. 
E = K.T @ F @ K # The essential matrix is just the same as the fundamental matrix when K = I

#E, mask = cv2.findEssentialMat(pts1,pts2,K,cv2.RANSAC) # alternatively, we may try to find essential matrix directly

In [4]:
# Sanity check of the essential matrix
import numpy as np

hx1=np.hstack((pts1,np.ones((2000,1))))
hx2=np.hstack((pts2,np.ones((2000,1))))

rint=np.random.randint(hx1.shape[0])
hx2[rint].T @ E @hx1[rint] # this should be almost zero

-4.017814525525409e-09

# Q1 (5 points) Compute a potential solution of R and t from E. Note that you will only get half of the points if you use cv2.decomposeEssentialMat

In [5]:
import numpy as np

# this function should be helpful. You probably want to call the function below instead of np.linalg.svd
def mySVD(E): # compute SVD E = U S V and enforcing det(U)=det(V)=1
    U,S,V = np.linalg.svd(E)
    detU=np.linalg.det(U)
    detV=np.linalg.det(V)
    U=U/detU
    V=V/detV
    S=S*detU*detV
    return U,S,V

def compute_one_R_and_t_from_E(E):
    # Input:
        # E: essential matrix
    # Output:
        # R: rotation matrix (3x3)
        # t: translation (3x1)
    pass

## Testing solution of Q.1

In [7]:
R,t = compute_one_R_and_t_from_E(E)

tx= np.array([[0,t[2],-t[1]],[-t[2],0,t[0]],[t[1],-t[0],0]])
R@tx-E # this should be almost 0

array([[-1.03683028e-06,  1.28662268e-05,  1.28662304e-05],
       [-1.28662299e-05, -5.18415112e-07, -5.18415249e-07],
       [ 1.28662272e-05,  5.18415023e-07,  5.18415160e-07]])

# Q2.a (5 points) For two lines ${\bf a_1} + \lambda_1 {\bf b_1}$ and ${\bf a_2} + \lambda_2 {\bf b_2}$ in the 3-D space parametrized by $\lambda_1$ and $\lambda_2$ (${\bf a_1}, {\bf a_2}, {\bf b_1}, {\bf b_2}$ are length-$3$ vectors). Find the intersecting point between the two lines (the mid point between the closest points of the two lines) by derivating the expressions of the optimum $\lambda_1$ and $\lambda_2$.

# Q2.b (5 points) Implement the solution of Q2.a by completing the function below



In [8]:
def computeIntersection(a1,b1,a2,b2):
    # Input:
        # a1: Nx3 matrix  (a1[i] = ith a1)
        # b1: Nx3 matrix  (b1[i] = ith b1)
        # a2: Nx3 matrix  (a2[i] = ith a2)
        # b2: Nx3 matrix  (b2[i] = ith b2)
        # N.B. for the ith pair of lines, line 1: a1[i]+lambda1[i] b1[i] and line 2: a2[i]+lambda2[i] b2[i]
    # Output:
        # points: Nx3 matrix (points[i] = the intersecting point for ith pair)
    pass

## Testing solution of Q2.b
#### We will take camera center of view 1 as origin, so ${\bf a_1} = {\bf 0}$, ${\bf b_1} = [x_1[0],x_1[1],1]^\top$
#### And ${\bf a_2} = {\bf t}$, $b_2 = R [x_2[0],x_2[1],0]^\top$



In [10]:
a1 = np.zeros((pts1.shape[0],3))
b1 = np.hstack((pts1,np.ones((pts1.shape[0],1))))
a2 = np.tile(t,(pts1.shape[0],1))
b2 = np.hstack((pts2,np.ones((pts2.shape[0],1))))
b2 = (R.T @ b2.T).T

Xs=computeIntersection(a1,b1,a2,b2) # 3D coordinates of points in the first camera view
Xps=(R@(Xs-t).T).T # 3D coordinates of points in the second camera view
Xs,Xps

(array([[  -0.28687998,    1.8775782 , -109.14884922],
        [  -0.23945037,    0.40417965, -109.63699108],
        [  -0.37123112,    0.47409453, -109.60449159],
        ...,
        [  -0.61579632,    0.89345589, -107.63380911],
        [  -0.84394685,    0.97585321, -107.6415475 ],
        [  -0.7574854 ,    1.42514355, -107.67678137]]),
 array([[  0.28687946,  -2.24669299, 105.02460402],
        [  0.23944984,  -2.73483479, 106.49800259],
        [  0.37123059,  -2.70233531, 106.42808771],
        ...,
        [  0.6157958 ,  -0.73165284, 106.00872628],
        [  0.84394633,  -0.73939123, 105.92632896],
        [  0.75748488,  -0.77462512, 105.47703862]]))

# Chirality

### Note that Xs and Xps are the points in the 3D space with camera centers as origins and with z-axis pointing from the camera centers to the objects. So the z-component (third column of Xs and Xps) should be both non-negative because the object points suppose to be in front of the cameras. But with only 1/4 of the chance you would be lucky. Because there are four possible combinations of R and t and only one is correct (satisfies chirality).

# Q2.c (4 points) Find the correct R and t by adjusting your solution in Q1. It is okay to provide a "buggy" solution that only works for the current dataset. Please redefine compute_one_R_and_t_from_E(E) below 

In [11]:
import numpy as np

# this function should be helpful. You probably want to function below instead of np.linalg.svd
def mySVD(E): # compute SVD E = U S V and enforcing det(U)=det(V)=1
    U,S,V = np.linalg.svd(E)
    detU=np.linalg.det(U)
    detV=np.linalg.det(V)
    U=U/detU
    V=V/detV
    S=S*detU*detV
    return U,S,V

def compute_one_R_and_t_from_E(E):
    # Input:
        # E: essential matrix
    # Output:
        # R: rotation matrix (3x3)
        # t: translation (3x1)
    pass

In [13]:
# rerun everything below

R,t = compute_one_R_and_t_from_E(E)

tx= np.array([[0,t[2],-t[1]],[-t[2],0,t[0]],[t[1],-t[0],0]])
rint=np.random.randint(hx1.shape[0])
hx2[rint].T @ R@tx @hx1[rint] # this should be almost zero


6.238334793184919e-07

In [14]:
a1 = np.zeros((pts1.shape[0],3))
b1 = np.hstack((pts1,np.ones((pts1.shape[0],1))))
a2 = np.tile(t,(pts2.shape[0],1))
b2 = np.hstack((pts2,np.ones((pts2.shape[0],1))))
b2 = (R.T @ b2.T).T

Xs=computeIntersection(a1,b1,a2,b2)
Xps=(R@(Xs-t).T).T 
Xs,Xps # note that third third columns should be positive

(array([[  0.28687998,  -1.8775782 , 109.14884922],
        [  0.23945037,  -0.40417965, 109.63699108],
        [  0.37123112,  -0.47409453, 109.60449159],
        ...,
        [  0.61579632,  -0.89345589, 107.63380911],
        [  0.84394685,  -0.97585321, 107.6415475 ],
        [  0.7574854 ,  -1.42514355, 107.67678137]]),
 array([[  0.28687946,  -2.24669299, 105.02460402],
        [  0.23944984,  -2.73483479, 106.49800259],
        [  0.37123059,  -2.70233531, 106.42808771],
        ...,
        [  0.6157958 ,  -0.73165284, 106.00872628],
        [  0.84394633,  -0.73939123, 105.92632896],
        [  0.75748488,  -0.77462512, 105.47703862]]))

In [15]:
# Sanity check: the projection to cameras should get back the original

print(pts1[0],Xs[0]/Xs[0,2]) # the numbers should match if you did correctly
print(pts2[0],Xps[0]/Xps[0,2])

[ 0.00262833 -0.017202  ] [ 0.00262834 -0.017202    1.        ]
[ 0.00273155 -0.02139206] [ 0.00273155 -0.02139206  1.        ]


In [16]:
#!pip install open3d # install open3d if you don't have it installed before
# visualize your point cloud

import open3d as o3d

cloud=o3d.geometry.PointCloud(o3d.utility.Vector3dVector(Xps)) 
o3d.visualization.draw_geometries([cloud])

# Q2.d (1 point): What is the object that you are seeing? Click the screen and move the mouse around and you can rotate it

# Q3 (10 points, extra credit): Capture two images and try to reconstruct 3D shape with the tools you developed.